In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns

In [2]:
df= sns.load_dataset('penguins')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


**ML SUPERVISADO TIPO CLASIFICACIÓN**

DATA SET DE CLASIFICACIÓN
TARGET = SPECIES 

In [3]:
df.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

In [4]:
df.shape

(344, 7)

¿Cómo imputamos la vez pasada? 
cuanto mejor imputemos, mejor predecirá el modelo.


### FUNCIONES SK LEARN

- make_pipeline
- StandardScaler
- MinMaxScaler
- LabelEncoder
- OneHotEncoder
- SimpleImputer


- Pipeline = tubería. Pasos ordenados donde cada unión entre tubos, tiene un filtro. Cada filtro es un paso intermedio. Todo entra por un punto y cuando sale, sale transformado como quieres. 
        Sirve para no tener que transformar y escalar los datos de test al comprobar el modelo. 
        Tardamos más en inicio pero agiliza operaciones futuras.
- Standard Scaler: resta la media y divide entre la desv estandar
- MinMaxScaler: escala para que los datos estén entre 0 y 1. Es más útil cuando necesitas los valores en un rango especifico. No es tan sensible a outliers.
- LabelEncoder: cambia los únicos de una columna por un valor numérico. 
- OneHotEncoder: versión pro del getdummies (es un método que no puedo guardar). Este es una clase qeu tiene . fit, puedo guardarlo y aplicarlo en cualquier dataset. Es similar. Sirve para dividir los únicos en columnas nuevas con valores  binarios que indican si la caracteristica se da o no.
- SimpleImputer: imputa Nan a lo que nosotros le pasemos, media, moda, mediana, o una función que le pase. 


In [20]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [6]:
# Realizamos las imputaciones

df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex'],
      dtype='object')

In [7]:
numeric_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
categorical_features = ['island','sex']
# sacamos species de categorical porque es la TARGET

- make_pipeline: Construye un pipeline con los estimadores. Metemos en una lista de tuplas:(  (lo que queremos que haga:'nombre', cómo funciona:(en este caso simple imputer utilizando la media(viene por defecto) yla moda)),

NO SE METE TUPLA




 )

In [8]:
X = df.drop(columns='species')
y = df['species']

In [28]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()

)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore'),
    
)

In [29]:
numeric_transformer

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [30]:
categorical_transformer

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [31]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numeric_transformer,numeric_features),
        ('cat',categorical_transformer,categorical_features)
        ]
)

# el num y cat en string es para verlo más abajo en model

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42) 
# no es necesario

In [33]:
model = make_pipeline(preprocessor, LogisticRegression() )

In [34]:
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['island', 'sex'])])),
                ('logisticregression', LogisticRegression())])

In [36]:
model.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['bill_length_mm',
                                                   'bill_depth_mm',
                                                   'flipper_length_mm',
                                                   'body_mass_g']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['island', 'sex'])])),
                ('logisticregression', LogisticRegression())])

In [39]:
x_transformed = preprocessor.fit_transform(X_train)
pd.DataFrame(x_transformed)
# pipelines devuelve arrays, no trabaja con df
# habría que poner nombres a las columnas(son las de x_train, las 4 primeras, y luego 2 de sex y 3 de isla)

,0,1,2,3,4,5,6,7,8
0,-1.224893,0.514609,-1.634660,-0.384484,0.0,1.0,0.0,0.0,1.0
1,0.385709,-1.004430,0.968023,1.114769,1.0,0.0,0.0,0.0,1.0
2,0.277129,-1.055065,1.319736,0.989831,1.0,0.0,0.0,0.0,1.0
3,1.453411,0.514609,-0.016776,-0.197077,0.0,1.0,0.0,0.0,1.0
4,0.602868,-0.903161,1.460422,1.302175,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
235,0.657158,0.615878,-0.438833,-0.446953,0.0,1.0,0.0,1.0,0.0
236,-0.772477,0.666513,-0.790547,-0.384484,0.0,0.0,1.0,0.0,1.0
237,-0.971540,0.058897,-0.157462,-0.571891,1.0,0.0,0.0,1.0,0.0
238,0.476192,-1.460142,0.616309,0.802424,1.0,0.0,0.0,1.0,0.0


In [40]:
X_train

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
31,Dream,37.2,18.1,178.0,3900.0,Male
245,Biscoe,46.1,15.1,215.0,5100.0,Male
277,Biscoe,45.5,15.0,220.0,5000.0,Male
165,Dream,52.0,18.1,201.0,4050.0,Male
250,Biscoe,47.3,15.3,222.0,5250.0,Male
...,...,...,...,...,...,...
188,Dream,47.6,18.3,195.0,3850.0,Female
71,Torgersen,39.7,18.4,190.0,3900.0,Male
106,Biscoe,38.6,17.2,199.0,3750.0,Female
270,Biscoe,46.6,14.2,210.0,4850.0,Female


In [41]:
x_transformed = preprocessor.fit_transform(X_test)
pd.DataFrame(x_transformed)

# salen las mismas columnas, sin errores de limpieza en los que podemos incurrir si no hacemos el pipeline

,0,1,2,3,4,5,6,7,8
0,1.357514,0.925346,-0.309453,-0.803596,0.0,1.0,0.0,0.0,1.0
1,0.265588,0.254144,-0.160858,-0.299667,0.0,1.0,0.0,1.0,0.0
2,0.514624,-1.965985,0.730712,0.456227,1.0,0.0,0.0,1.0,0.0
3,0.265588,-0.365427,-0.680940,-1.181543,0.0,1.0,0.0,1.0,0.0
4,0.878599,-1.501307,0.210630,0.550714,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
99,-0.328266,0.925346,-0.383750,-0.236676,0.0,0.0,1.0,0.0,1.0
100,-1.094529,-0.158903,-1.423914,-1.276030,1.0,0.0,0.0,1.0,0.0
101,-1.803323,0.563930,-1.201022,-1.087056,0.0,0.0,1.0,1.0,0.0
102,-0.998746,-0.158903,-0.903832,-1.622481,0.0,0.0,1.0,1.0,0.0
